In [36]:
from DatabaseConnections import *

In [59]:
db = PostgresDb('Dotdevpgsql01.dot.nycnet', 'CRASHDATA')

User name (CRASHDATA):shostetter
Password (CRASHDATA):········


In [60]:
class Graph:
    def __init__(self):
        self.edges = defaultdict(list) # edges[segmentid] = [nodefrom, node]
        self.nodes = defaultdict(dict) # nodes[node1] = {node: dist}
        self.edge_from_nodes = defaultdict(list) # edges[(nodefrom, node)] = segmentid
    
    def graph(self, data):
        for row in data:
            street, segmentid, trafdir, nodefrom, nodeto, feet = row
            self.edges[segmentid].append(nodefrom)
            self.edges[segmentid].append(nodeto)
            if trafdir == 'W': # from->to
                self.nodes[nodefrom][nodeto] = feet
                self.edge_from_nodes[(nodefrom, nodeto)] = segmentid
            elif trafdir == 'A"': # to->from
                self.nodes[nodeto][nodefrom] = feet
                self.edge_from_nodes[(nodeto, nodefrom)] = segmentid
            else:
                self.nodes[nodefrom][nodeto] = feet
                self.nodes[nodeto][nodefrom] = feet
                self.edge_from_nodes[(nodefrom, nodeto)] = segmentid
                self.edge_from_nodes[(nodeto, nodefrom)] = segmentid

# g = Graph()
# g.graph(data)
# print g.nodes['0046125M']
# print g.edges['0175097']
# print g.edge_from_nodes[('0046125M', '0082670M')]

In [61]:
def dijkstra(graph,v):
    dist_so_far = {}
    dist_so_far[v] = (0,[v])
    left=[v]
    while len(left)>0:
        w= left.pop(0)
        weight, path =dist_so_far[w]
        for x in graph[w]:
            path2=path+[x]
            weight2=min([weight,graph[w][x]])
            if x in dist_so_far:
                if weight2<dist_so_far[x][0]:
                    dist_so_far[x]=(weight2,path2)
                    if x not in left:
                        left.append(x)
            else:
                dist_so_far[x]=(weight2,path2)
                if x not in left:
                    left.append(x)
    return dist_so_far

@timeDec
def dijkstraPath(graph, start, end):
    dPath = dijkstra(graph, start)
    if end in dPath:
        return dPath[end][1]
    else:
        return None

In [62]:
def get_closest_segment(db, x, y, lion='lion'):
    seg, col = db.query("""
        SELECT DISTINCT segmentid from ( SELECT segmentid FROM {2}
        WHERE rb_layer in ('R', 'B', 'N') and featuretyp in ('0', '6', 'C')
        ORDER BY geom <-> st_setsrid(st_makepoint({0}, {1}),2263)
        LIMIT 1) s;""".format(x,y, lion))
    segmentid = seg[0][0]
    return segmentid

def get_start_end_node(db, x, y, lion='lion'):
    segmentid = get_closest_segment(db, x, y, lion)
    data, cols = db.query("""
        SELECT node, x, y from (
            select nodeidfrom||nodelevelf node, xfrom::float x, yfrom::float y from {3} where segmentid = '{0}'
            union select nodeidto||nodelevelt node, xto::float, yto::float from {3} where segmentid = '{0}') seg 
            ORDER BY st_setsrid(st_makepoint(x,y), 2263) <-> st_setsrid(st_makepoint({1}, {2}),2263)
        LIMIT 1
    """.format(segmentid, x, y, lion))
    node, x, y = data[0]
    return node

x, y = 981569.692482606, 195645.001810625
print get_closest_segment(db, x, y)
print get_start_end_node(db, x, y)


0023137
0015228M


In [63]:
def get_segs_from_path(path_list):
    segs = list()
    s = path_list[0]
    for node in path_list[1:]:
        seg = g.edge_from_nodes[(s, node)]
        if seg ==[]:
            print s, node
        s = node
        segs.append(seg)
    return segs

def make_fc_from_segs(segs, fc_name='working.TEST', lion='lion'):
    db.query("drop table if exists {0};".format(fc_name))
    db.query("""
        create table {0} as 
        select street, segmentid, nodeidfrom, nodeidto, geom
        from {2}
        where segmentid in ({1})
    """.format(fc_name, str(segs)[1:-1], lion))
    print 'New FC {0} to review...'.format(fc_name)   
    
    

### Sample table for graphing

In [64]:
query_to_table(db, """
    select distinct street, segmentid, trafdir, nodeidfrom||nodelevelf nodefrom, nodeidto||nodelevelt nodeto, 
    st_length(geom) as feet
    from {0} where rb_layer in ('R', 'B', 'N') and featuretyp in ('0', '6', 'C') limit 10""".format('lion'))

,street,segmentid,trafdir,nodefrom,nodeto,feet
0,1 AVENUE,0017690,T,0012098M,0012097M,301.533481
1,1 AVENUE,0017692,T,0012099M,0012098M,275.179907
2,1 AVENUE,0017699,T,0012105M,0012099M,260.605618
3,1 AVENUE,0017812,T,0012165M,0012105M,259.585744
4,1 AVENUE,0017815,T,0012166M,0012165M,260.285818
5,1 AVENUE,0017817,T,0012168M,0012166M,263.179966
6,1 AVENUE,0017821,T,0012170M,0012168M,257.621150
7,1 AVENUE,0017826,T,0012174M,0012170M,259.113893
8,1 AVENUE,0017828,T,0012175M,0012174M,258.192743
9,1 AVENUE,0017928,T,0012227M,0012175M,261.700810


## Graph LION network

In [65]:
data, cols = db.query("""
        select distinct street, segmentid, trafdir, nodeidfrom||nodelevelf nodefrom, nodeidto||nodelevelt nodeto, 
        st_length(geom) as feet
        from lion where rb_layer in ('R', 'B', 'N') and featuretyp in ('0', '6', 'C')
    """)
g = Graph()
g.graph(data)

### store past paths between segments/nodes so they dont need to be rerouted

should use segments to cut down on the number of possible paths... TODO

In [66]:
mapped = defaultdict(list) # mapped[(start_node, end_node)] = [path]

def check_mapped(db, graph, start_cords, end_cords, lion='lion'):
    # get nodes from cords
    start = get_start_end_node(db, start_cords[0], start_cords[1], lion)
    end = get_start_end_node(db, end_cords[0], end_cords[1], lion)
    # check if path exists
    if (start, end) in mapped.keys():
        print 'Path found\ndictionary ({})'.format(len(mapped.keys()))
        return mapped[(start, end)] # if yes return path
    else:
        mapped[(start, end)] = dijkstraPath(graph, start, end)
        print 'Path not found\nAdding to dictionary ({})'.format(len( mapped.keys()))
        return mapped[(start, end)] # if no map and add path


In [68]:
@timeDec
def test(graph, data):
    # sample x/y pairs Lower MN to Bklyn
    start = (981569.692482606, 195645.001810625)
    end = (993239.543786891, 199024.085521124)
    path = check_mapped(db, graph, start, end, 'lion')
    return get_segs_from_path(path)

# first run
segs = test(g.nodes, data)
# second run 64% faster
segs = test(g.nodes, data)
make_fc_from_segs(segs, 'working.TEST', 'lion')
# db.query("drop table if exists working.TEST;")

'dijkstraPath' 2.30 sec
Path not found
Adding to dictionary (1)
'test' 3.59 sec
Path found
dictionary (1)
'test' 1.27 sec
Update sucessfull
Update sucessfull
New FC working.TEST to review...





## Test on DCAS GPS data 
#### Single vehicle single day (small sample n =223)

In [35]:
@timeDec
def alttest(graph, data):
    # sample x/y pairs Lower MN to Bklyn
    start = (981569.692482606, 195645.001810625)
    end = (993239.543786891, 199024.085521124)
    # find the best fit nodes at each end
    start_node = get_start_end_node(db, start[0], start[1])
    end_node = get_start_end_node(db, end[0], end[1])
    # get shortest path in nodes
    path =  dijkstraPath(graph, start_node, end_node)
    return get_segs_from_path(path)


# segs = alttest(g.nodes, data)
# make_fc_from_segs(segs, fc_name='working.TEST')
# # db.query("drop table if exists working.TEST;")

<img src="Capture.jpg">